# Tutorial for Introduction to ML Lecture

version 0.1, September 2023

Bryan Scott, CIERA/Northwestern

## Problem 1: Bayes Classifiers

A good starting point for Machine Learning is the Bayes classifier. The basic idea is to assign the most probable label to each data point using Bayes theorem, we take:

$$
p(y | x_n) \propto p(y)p(x_i, ..., x_n | y)
$$

where y is a label for a data point and the $x_n$ are the features of the data that we want to use to classify each data point. A $\textit{Naive} Bayes$ classifier makes an important simplifying assumptions that gives it the name - it assumes that the conditional probabilities are independent, $p(x_i, ..., x_n | y) = p(x_i|y)... p(x_n | y)$. That is, the probability of observing any individual feature doesn't depend on any of the other features. Our task is to construct this classifier from a set of examples we've observed previously and compare it to new data. 

### Part 0: Load and split the data

In [3]:
import pandas as pd
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt

In [4]:
# lsst_data[0:1000].to_csv('session_19_DC2_subset.csv')

### Loading and splitting the data. 

Read in the data, then start by selecting the id, fluxes, and object truth type in the lsst data file you've been provided. 

Once you have selected those, randomly split the data into two arrays, one containing 80% of the data, and a second array containing 20% of the data. 

In [5]:
from sklearn.model_selection import train_test_split

cols = [
    'id',
    'flux_g',
    'flux_i',
    'flux_r',
    'flux_u',
    'flux_y',
    'flux_z',
    'truth_type'
]

lsst_data = pd.read_csv('./simulated_extragalactic_data.csv', usecols=cols) #path to your data
unique_values = lsst_data['truth_type'].unique()
print(unique_values)

lsst_data_filtered = lsst_data[lsst_data['truth_type'] != 3]

# Split the DataFrame into 80% training and 20% testing
train_df, test_df = train_test_split(lsst_data_filtered, test_size=0.2, random_state=36)

print(len(train_df[train_df['truth_type'] == 1]))
print(len(train_df[train_df['truth_type'] == 2]))
print(len(train_df[train_df['truth_type'] == 1]) + len(train_df[train_df['truth_type'] == 2]))
print(len(train_df))



[1 2 3]
655
143
798
798


### Part 1: Estimate Class Frequency in the training set

One of the ingredients in our classifier is p(y), the unconditional class probabilities. 

We can get this by counting the number of rows belonging to each class in train_data and dividing by the length of the training data set. 

In [6]:
def estimate_class_probabilities(x_train):
    """
    Computes unconditional class probabilities. 
     
    Args:
        x_train (array): training data for the classifier
 
    Returns:
        ints p1, p2: unconditional probability of an element of the training set belonging to class 1
    """
    unique_values = x_train['truth_type'].unique()
    
    ps = []
    for val in unique_values:
        p = len(x_train[x_train['truth_type'] == val]) / len(x_train)
        ps.append(p)
        
    return ps



estimate_class_probabilities(train_df)



[0.8208020050125313, 0.17919799498746866]

### Part 2:  Feature Likelihoods

We are assuming that the relationship between the classes and feature probabilities are related via:

$p(x_i, ..., x_n | y) =  p(x_i|y)... p(x_n | y)$

however, we still need to make an assumption about the functional form of the $p(x_i | y)$. As a simple case, we will assume $p(x_i | y)$ follows a Gaussian distribution given by:

$$
p(x_i | y) = \frac{1}{\sqrt{2 \pi \sigma_y}} \exp{\left(-\frac{(x_i - \mu)^2}{\sigma_y^2}\right)}
$$

and we will make a maximum likelihood estimate of $\mu$ and $\sigma_y$ from the data. This means using empirical estimates $\bar{x}$ and $\hat{\sigma}$ as estimators of the true parameters $\mu$ and $\sigma_y$. 

Write a fitting function that takes the log of the fluxes and returns an estimate of the parameters of the per-feature likelihoods for each class.

In [7]:
def per_feature_likelihood_parameters(x_train, label, flux_band):
    """"
    Computes MAP estimates for the class conditional likelihood. 
     
    Args:
        x_train (array or pd series): training data for the classifier
        label (df): training labels for the classifier 
        flux_band (str): which flux band
 
    Returns:
        means, stdevs (array): MAP estimates of the Gaussian conditional probability distributions for a specific class
    """
    
    means = []
    stdevs = []
    
    #for flux in ['flux_g','flux_i','flux_r','flux_u','flux_y','flux_z',]:
    log_flux = np.log(x_train[flux_band])

    means = [np.mean(log_flux[label==1]), np.mean(log_flux[label==2])]
    stdevs = [np.std(log_flux[label==1]), np.std(log_flux[label==2])]


    return means, stdevs


In [8]:
per_feature_likelihood_parameters(train_df, train_df['truth_type'], 'flux_u')



([2.7108576412688565, 5.111269509911686],
 [1.3672672221282172, 3.9719110841555776])

### Part 3: MAP Estimates of the Class Probabilities

Now that we have the unconditional class probabilities and the parameters of the per feature likelihoods in hand, we can put this all together to build the classifier. Use the methods you have already written to write a function that takes in the training data and returns fit parameters. Once you have done that, write a method that takes the fit parameters as an argument and predicts the class of new (and unseen) data. 

In [9]:
# build the classifier

# solved 

def fit(x_train, flux_band):
    """"
    Convenience function to perform fitting on the training data
     
    Args:
        x_train (array or pd series): training data for the classifier
        flux_band (str): which flux band

    Returns:
        p1, p2, class_1_mean, class_2_mean, class_1_std, class_2_std: see documentation for per_feature_likelihood_parameters
    """
    p1, p2 = estimate_class_probabilities(x_train)
    means, stdevs = per_feature_likelihood_parameters(x_train, x_train['truth_type'], flux_band)
    class_1_mean, class_2_mean = means[0], means[1]
    class_1_std, class_2_std = stdevs[0], stdevs[1]
    
    return p1, p2, class_1_mean, class_2_mean, class_1_std, class_2_std


In [10]:
fit(train_df, 'flux_u')

(0.8208020050125313,
 0.17919799498746866,
 2.7108576412688565,
 5.111269509911686,
 1.3672672221282172,
 3.9719110841555776)

In [11]:
def predict(x_test, class_probability, class_means, class_dev):
    """"
    Predict method
     
    Args:
        x_test (array): data to perform classification on
        class_probability (array): unconditional class probabilities
        class_means, class_dev (arrays): MAP estimates produced by the fit method
 
    Returns:
        predict_List (list): class membership predictions
    """
    
    # compute probabilities of an element of the test set belonging to class 1 or 2
    
    probs = []
    for ii in range(0, 2):
        p = class_probability[ii]
        mean = class_means[ii]
        stdev = class_dev[ii]

        
        gaussian_prob = 1/(np.sqrt(2*np.pi*stdev)) * np.exp(-(x_test - mean)**2./stdev**2.)
        probs.append(gaussian_prob)
    
    
    predict_list = []
    for ii in range(0, len(probs[0])):
        if probs[0][ii] > probs[1][ii]:
            predict_list.append(1)
        else:
            predict_list.append(2)
            
    
        
        
    
    
    return predict_list



In [12]:
fits = fit(train_df, 'flux_u')
print(predict(np.array(test_df['flux_u']), fits[0:2],  fits[2:4], fits[4:6]))

[2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2, 1, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


### Part 4: Metrics

After creating a classifier, you now want to evaluate it in terms of how often it correctly and incorrectly classifies the objects in your training set. To do this, we'll design a confusion matrix. A confusion matrix is a matrix whose entries are the counts of the predicted vs actual class. For example, the first entry is the count of objects that are predicted to be of class 1 and actually are of class 1 and so on, while the off-diagonal elements would be instances of class 1 that are predicted to be of class 2, and instances of class 2 that are predicted to be of class 1. 

In [37]:
def plot_confusion_matrix(df_confusion, cmap=):
    """
    
    Convenience function to plot the confusion matrix from a pd.crosstab object. Hint: use plt.matshow and choose a sensible color map.
    
    Args:
        df_confusion (pd.crosstab): A pd.crosstab object.
        
    Returns:
        null 
    """
    
    
    plt.matshow()


## Problem 2: The Cramer-Rao bound (pen & paper, challenging, optional)

As we saw in the lecture, the Cramer-Rao bound is an important result in statistics that has intuitive consequences for many applied problems in ML. The proof of the Cramer-Rao bound can be insightful to work through. 

The starting point for the proof of the bound is the Cauchy-Schwarz inequality, which can be used to show that:

$$
[Cov(U, V)]^2 \le Var(U)Var(V)
$$

Starting from the definitions that U = T(X), where T(X) is an estimator of some parameter $\theta$ of the distribution $f(X|\theta)$ from which the data is sampled, and V = $\frac{\partial}{\partial \theta} log f(X |\theta)$. Use the Cauchy-Schwarz inequality to show the Cramer-Rao bound for these choices of U and V. 

$\textit{Hint:}$ you will need the fact that the $\mathbb{E}(V) = 0$, where $\mathbb{E}$ is the expectation of a random variable.